In [1]:
import json
import yaml
import langchain

In [2]:
# client = openai.OpenAI(api_key=OPENAI_API_KEY)
#         response = client.chat.completions.create(
#             model="gpt-4.1-nano",
#             # model = "gpt-4.1-nano-2025-04-14",
#             messages=[
#                 {"role": "system", "content": "You are a search query generator that creates optimized search queries for finding job candidates."},
#                 {"role": "user", "content": prompt}
#             ],
#             max_tokens=300,
#             temperature=0.3
#         )
        
#         search_query = response.choices[0].message.content

In [3]:
import pandas as pd
df = pd.read_csv("labeled_conversations.csv")
df.head(10)

,conversation_id,conversation,profanity,sensitive_data_compliance
0,742147aa-a2df-4429-859e-05b4a7875563,"[{""speaker"": ""Agent"", ""text"": ""Hello, is this ...",Not Found,Not Found
1,b76917d8-cb80-481a-84d5-25de34e98f95,"[{""speaker"": ""Agent"", ""text"": ""Good afternoon,...",Not Found,Not Found
2,63803395-1ed2-405c-afdc-66ed0f48cd45,"[{""speaker"": ""Agent"", ""text"": ""Hello, this is ...",Not Found,Not Found
3,a73d777f-368a-46b5-9d59-0aa5829b4427,"[{""speaker"": ""Agent"", ""text"": ""Hello, is this ...",Not Found,Not Found
4,d7bbea61-d739-43fb-a198-ced1b59f9491,"[{""speaker"": ""Agent"", ""text"": ""Hello, this is ...",Found,Not Found
5,0b6979e4-8c05-49e1-b7a7-94d85a627df5,"[{""speaker"": ""Agent"", ""text"": ""Hello, this is ...",Not Found,Not Found
6,928acb70-fa80-4532-a0c0-2234bf236e17,"[{""speaker"": ""Agent"", ""text"": ""Hello, this is ...",Found,Not Found
7,b4dcda70-5083-4dc4-96a7-ce6d5e92c2a2,"[{""speaker"": ""Agent"", ""text"": ""Hello, this is ...",Not Found,Not Found
8,2f279983-ea96-4c10-a09d-4b607de21a38,"[{""speaker"": ""Agent"", ""text"": ""Hi, this is Mik...",Found,Violation
9,3f9a4a6a-2d53-4391-95c9-44306989d6ce,"[{""speaker"": ""Agent"", ""text"": ""Hello, is this ...",Not Found,Not Found


In [4]:
df["conversation"][0]

'[{"speaker": "Agent", "text": "Hello, is this Mark Johnson? This is Sarah calling from XYZ Collections. How are you today?", "stime": 0, "etime": 8}, {"speaker": "Customer", "text": "Hi Sarah, yes this is Mark. I\'m doing okay, thanks.", "stime": 8.5, "etime": 12}, {"speaker": "Agent", "text": "I\'m glad to hear that, Mark. I\'m reaching out regarding your overdue balance with ABC Credit Union. Can I get your account number to look it up?", "stime": 11, "etime": 21}, {"speaker": "Customer", "text": "Sure, it\'s 123456789.", "stime": 22, "etime": 25}, {"speaker": "Agent", "text": "Thank you for that. Now, your current balance is $500. Would you like to discuss payment options?", "stime": 24, "etime": 34}, {"speaker": "Customer", "text": "I was thinking of paying half today and the rest next month.", "stime": 35, "etime": 42}, {"speaker": "Agent", "text": "That sounds reasonable. I can help facilitate that. Just to clarify, can I confirm your mailing address?", "stime": 41, "etime": 51}

In [5]:
type(df["conversation"][0])

str

In [6]:
import pickle as pk


In [7]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   conversation_id            250 non-null    object
 1   conversation               250 non-null    object
 2   profanity                  250 non-null    object
 3   sensitive_data_compliance  250 non-null    object
dtypes: object(4)
memory usage: 7.9+ KB


In [ ]:
import pandas as pd
import numpy as np
import json
import re
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import warnings
warnings.filterwarnings('ignore')

class CallAnalysisModelBuilder:
    def __init__(self, df_path=None, df=None):
        """
        Initialize the model builder
        Args:
            df_path: Path to CSV file containing the dataframe
            df: Pandas dataframe (if already loaded)
        """
        if df is not None:
            self.df = df
        elif df_path:
            self.df = pd.read_csv(df_path)
        else:
            raise ValueError("Either df_path or df must be provided")
        
        # Initialize components
        self.profanity_vectorizer = None
        self.sensitive_vectorizer = None
        self.profanity_model = None
        self.sensitive_model = None
        self.label_encoder = LabelEncoder()
        
    def preprocess_conversation_text(self, conversation_str):
        """
        Extract and preprocess text from conversation JSON string
        Args:
            conversation_str: JSON string containing conversation data
        Returns:
            combined_text: Preprocessed text from all speakers
        """
        try:
            # Parse JSON string
            conversation_list = json.loads(conversation_str)
            
            # Extract all text and combine
            all_text = []
            for item in conversation_list:
                if 'text' in item:
                    all_text.append(item['text'])
            
            # Combine all text
            combined_text = " ".join(all_text)
            
            # Basic preprocessing
            combined_text = combined_text.lower()
            combined_text = re.sub(r'[^\w\s]', ' ', combined_text)  # Remove punctuation
            combined_text = re.sub(r'\s+', ' ', combined_text).strip()  # Remove extra spaces
            
            return combined_text
            
        except (json.JSONDecodeError, KeyError) as e:
            print(f"Error processing conversation: {e}")
            return ""
    
    def extract_agent_customer_text(self, conversation_str):
        """
        Extract separate agent and customer text for sensitive data analysis
        """
        try:
            conversation_list = json.loads(conversation_str)
            agent_text = []
            customer_text = []
            
            for item in conversation_list:
                if 'text' in item and 'speaker' in item:
                    text = item['text'].lower()
                    if item['speaker'].lower() == 'agent':
                        agent_text.append(text)
                    else:
                        customer_text.append(text)
            
            return " ".join(agent_text), " ".join(customer_text)
        except:
            return "", ""
    
    def prepare_data(self):
        """
        Prepare data for training both models
        """
        print("Preprocessing conversation data...")
        
        # Extract text from conversations
        self.df['processed_text'] = self.df['conversation'].apply(self.preprocess_conversation_text)
        
        # Extract agent and customer text separately for sensitive data analysis
        agent_customer_data = self.df['conversation'].apply(self.extract_agent_customer_text)
        self.df['agent_text'] = [item[0] for item in agent_customer_data]
        self.df['customer_text'] = [item[1] for item in agent_customer_data]
        
        # Combine agent and customer text for sensitive data analysis
        self.df['combined_sensitive_text'] = self.df['agent_text'] + " " + self.df['customer_text']
        
        # Encode labels (assuming 'found'/'not found' format)
        self.df['profanity_label'] = self.label_encoder.fit_transform(self.df['profanity'])
        self.df['sensitive_label'] = self.label_encoder.fit_transform(self.df['sensitive_data_compliance'])
        
        print(f"Data shape: {self.df.shape}")
        print(f"Profanity distribution: {self.df['profanity'].value_counts()}")
        print(f"Sensitive data distribution: {self.df['sensitive_data_compliance'].value_counts()}")
        
    def create_profanity_keywords(self):
        """
        Create profanity detection patterns and keywords
        """
        profanity_patterns = [
            r'\b(damn|hell|shit|fuck|ass|bitch|crap)\b',
            r'\b(stupid|idiot|moron|dumb)\b',
            r'\b(wtf|omg|goddamn)\b'
        ]
        return profanity_patterns
    
    def create_sensitive_data_keywords(self):
        """
        Create sensitive data detection patterns
        """
        sensitive_patterns = [
            r'\b\d{3}-\d{2}-\d{4}\b',  # SSN pattern
            r'\b\d{4}\s*\d{4}\s*\d{4}\s*\d{4}\b',  # Credit card pattern
            r'\baccount\s*number\b',
            r'\bbalance\b',
            r'\bssn\b|social\s*security',
            r'\bdate\s*of\s*birth\b|dob\b',
            r'\baddress\b',
            r'\bverification\b|verify\b'
        ]
        return sensitive_patterns
    
    def build_profanity_model_sklearn(self):
        """
        Build profanity detection model using scikit-learn
        """
        print("Building profanity detection model (scikit-learn)...")
        
        # Prepare features
        X = self.df['processed_text']
        y = self.df['profanity_label']
        
        # Create TF-IDF features
        self.profanity_vectorizer = TfidfVectorizer(
            max_features=1000,
            ngram_range=(1, 2),
            stop_words='english'
        )
        
        X_tfidf = self.profanity_vectorizer.fit_transform(X)
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X_tfidf, y, test_size=0.2, random_state=42, stratify=y
        )
        
        # Train model
        self.profanity_model = RandomForestClassifier(
            n_estimators=100,
            random_state=42,
            class_weight='balanced'
        )
        
        self.profanity_model.fit(X_train, y_train)
        
        # Evaluate
        y_pred = self.profanity_model.predict(X_test)
        print("Profanity Model Performance:")
        print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
        print("\nClassification Report:")
        print(classification_report(y_test, y_pred))
        
    def build_sensitive_data_model_tensorflow(self):
        """
        Build sensitive data compliance model using TensorFlow
        """
        print("Building sensitive data compliance model (TensorFlow)...")
        
        # Prepare features
        X = self.df['combined_sensitive_text']
        y = self.df['sensitive_label']
        
        # Create TF-IDF features
        self.sensitive_vectorizer = TfidfVectorizer(
            max_features=1500,
            ngram_range=(1, 3),
            stop_words='english'
        )
        
        X_tfidf = self.sensitive_vectorizer.fit_transform(X).toarray()
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X_tfidf, y, test_size=0.2, random_state=42, stratify=y
        )
        
        # Build neural network
        model = Sequential([
            Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
            Dropout(0.3),
            Dense(256, activation='relu'),
            Dropout(0.3),
            Dense(128, activation='relu'),
            Dropout(0.2),
            Dense(1, activation='sigmoid')
        ])
        
        model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        # Train model
        history = model.fit(
            X_train, y_train,
            epochs=50,
            batch_size=32,
            validation_split=0.2,
            verbose=1
        )
        
        # Evaluate
        test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
        print(f"Sensitive Data Model Performance:")
        print(f"Test Accuracy: {test_accuracy:.4f}")
        
        self.sensitive_model = model
        
    def save_models(self):
        """
        Save trained models and vectorizers
        """
        print("Saving models...")
        
        # Save profanity model (scikit-learn)
        with open('profanity_model.pkl', 'wb') as f:
            pickle.dump({
                'model': self.profanity_model,
                'vectorizer': self.profanity_vectorizer,
                'label_encoder': self.label_encoder
            }, f)
        
        # Save sensitive data model (TensorFlow)
        self.sensitive_model.save('sensitive_model.h5')
        
        with open('sensitive_vectorizer.pkl', 'wb') as f:
            pickle.dump(self.sensitive_vectorizer, f)
        
        print("Models saved successfully!")
        print("Files created:")
        print("- profanity_model.pkl")
        print("- sensitive_model.h5")
        print("- sensitive_vectorizer.pkl")
    
    def train_models(self):
        """
        Complete training pipeline
        """
        self.prepare_data()
        self.build_profanity_model_sklearn()
        self.build_sensitive_data_model_tensorflow()
        self.save_models()



# Example usage and testing
if __name__ == "__main__":
    # Example of how to use the classes
    
    print("🔥 CALL ANALYSIS ML MODEL TRAINING 🔥")
    print("="*60)
    
    # 1. Training phase (run once)
    print("=== TRAINING PHASE ===")
    
    # Load your dataframe (replace with your actual data path)
    try:
        # Uncomment and modify this line to load your actual data:
        # df = pd.read_csv('your_dataframe.csv')
        
        # For demonstration, create a sample dataframe structure
        sample_data = {
            'conversation_id': ['call_001', 'call_002', 'call_003', 'call_004'],
            'conversation': [
                '[{"speaker": "Agent", "text": "Hello sir", "stime": 0, "etime": 8}]',
                '[{"speaker": "Customer", "text": "This is damn frustrating", "stime": 0, "etime": 5}]',
                '[{"speaker": "Agent", "text": "Your balance is $500, SSN 123-45-6789", "stime": 0, "etime": 8}]',
                '[{"speaker": "Agent", "text": "Thank you for calling", "stime": 0, "etime": 5}]'
            ],
            'profanity': ['not found', 'found', 'not found', 'not found'],
            'sensitive_data_compliance': ['found', 'not found', 'found', 'not found']
        }
        df = pd.DataFrame(sample_data)
        print(f"📊 Sample data created with {len(df)} rows")
        
        # UNCOMMENT THESE LINES TO TRAIN WITH YOUR REAL DATA:
        # builder = CallAnalysisModelBuilder(df=df)
        # builder.train_models()
        
        print("\n⚠️  TO TRAIN WITH YOUR REAL DATA:")
        print("1. Load your CSV: df = pd.read_csv('your_data.csv')")
        print("2. Uncomment: builder = CallAnalysisModelBuilder(df=df)")
        print("3. Uncomment: builder.train_models()")
        
    except Exception as e:
        print(f"❌ Error in training phase: {e}")
    
    # # 2. Prediction phase (use after training)
    # print("\n" + "="*60)
    # print("=== PREDICTION PHASE DEMO ===")
    
    # # Example JSON input
    # sample_conversation = '''[
    #     {"speaker": "Agent", "text": "Hello, is this Mark Johnson? This is Sarah calling from XYZ Collections. How are you today?", "stime": 0, "etime": 8},
    #     {"speaker": "Customer", "text": "Hi Sarah, yes this is Mark. I'm doing okay, thanks.", "stime": 8.5, "etime": 12},
    #     {"speaker": "Agent", "text": "Thank you for that. Now, your current balance is $500. Would you like to discuss payment options?", "stime": 24, "etime": 34}
    # ]'''
    
    # print("📝 Sample conversation loaded")
    # print("💡 After training, you can use:")
    # print()
    # print("predictor = CallAnalysisPredictor()")
    # print("predictor.load_models()")
    # print("results = predictor.predict_both(sample_conversation)")
    # print()
    print("Expected output files after training:")
    print("📦 profanity_model.pkl")
    print("📦 sensitivedata_model.pkl") 
    print("📦 profanity_vectorizer.pkl")
    print("📦 sensitive_vectorizer.pkl")
    print("📦 label_encoder.pkl")
    
    print("\n" + "="*60)
    print("🚀 Ready to train your models!")

🔥 CALL ANALYSIS ML MODEL TRAINING 🔥
=== TRAINING PHASE ===
📊 Sample data created with 4 rows

⚠️  TO TRAIN WITH YOUR REAL DATA:
1. Load your CSV: df = pd.read_csv('your_data.csv')
2. Uncomment: builder = CallAnalysisModelBuilder(df=df)
3. Uncomment: builder.train_models()

=== PREDICTION PHASE DEMO ===
📝 Sample conversation loaded
💡 After training, you can use:

predictor = CallAnalysisPredictor()
predictor.load_models()
results = predictor.predict_both(sample_conversation)

Expected output files after training:
📦 profanity_model.pkl
📦 sensitivedata_model.pkl
📦 profanity_vectorizer.pkl
📦 sensitive_vectorizer.pkl
📦 label_encoder.pkl

🚀 Ready to train your models!


In [11]:
df = pd.read_csv("labeled_conversations.csv")
builder = CallAnalysisModelBuilder(df=df)
builder.train_models()

Preprocessing conversation data...
Data shape: (250, 10)
Profanity distribution: profanity
Not Found    211
Found         39
Name: count, dtype: int64
Sensitive data distribution: sensitive_data_compliance
Not Found    235
Violation     10
Found          5
Name: count, dtype: int64
Building profanity detection model (scikit-learn)...
Profanity Model Performance:
Accuracy: 1.0000

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        42

    accuracy                           1.00        50
   macro avg       1.00      1.00      1.00        50
weighted avg       1.00      1.00      1.00        50

Building sensitive data compliance model (TensorFlow)...
Epoch 1/50


2025-09-04 21:05:24.714872: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.8750 - loss: 0.5786 - val_accuracy: 0.9500 - val_loss: 0.3883
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9375 - loss: 0.1845 - val_accuracy: 0.9500 - val_loss: 0.0455
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9375 - loss: -0.1369 - val_accuracy: 0.9500 - val_loss: 0.0147
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9375 - loss: -0.3368 - val_accuracy: 0.9500 - val_loss: 0.0225
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9375 - loss: -0.6615 - val_accuracy: 0.9500 - val_loss: 0.0292
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9375 - loss: -1.3161 - val_accuracy: 0.9500 - val_loss: 0.0270
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9375 - loss: -2.4190 - val_accuracy: 0.9500 - val_loss: 0.0099
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9375 - loss: -3.9020 - val_accuracy: 0.9500 - val_loss: 0.0023
Epoch

Sensitive Data Model Performance:
Test Accuracy: 0.9400
Saving models...
Models saved successfully!
Files created:
- profanity_model.pkl
- sensitive_model.h5
- sensitive_vectorizer.pkl


In [22]:
predictor = CallAnalysisPredictor()
predictor.load_models()

# Your JSON conversation string
conversation_json = '[{"speaker":"Agent","text":"Hello, this is Mark calling from XYZ Collections. How are you today?","stime":0,"etime":7},{"speaker":"Customer","text":"I amm okay, just busy. What is this about?","stime":8,"etime":12},{"speaker":"Agent","text":"I understand, I will be brief. I\'m calling regarding your outstanding balance with Quick Loans. Can you confirm your name for me?","stime":11,"etime":20},{"speaker":"Customer","text":"Sure, it\'s Lisa Green.","stime":21,"etime":24},{"speaker":"Agent","text":"Thank you, Lisa. You currently have a balance of $450. How would you like to proceed with the payment?","stime":23,"etime":32},{"speaker":"Customer","text":"I need more information before I decide.","stime":33,"etime":36},{"speaker":"Agent","text":"Of course! Your last payment was made in January, and your balance has been accumulating since then. Would you like to set up a payment plan?","stime":35,"etime":45},{"speaker":"Customer","text":"I might consider that. What are the options?","stime":46,"etime":50},{"speaker":"Agent","text":"We can do weekly or monthly payments. For example, a monthly payment of $150 would work for you.","stime":49,"etime":58},{"speaker":"Customer","text":"That sounds reasonable, but I need to think it over.","stime":59,"etime":62},{"speaker":"Agent","text":"I understand. Please let me know if you have any questions. Can I count on you to respond by the end of the week?","stime":63,"etime":72},{"speaker":"Customer","text":"Yes, I\'ll get back to you. Thank you.","stime":71,"etime":74},{"speaker":"Agent","text":"You\'re welcome, Lisa. Have a great day!","stime":75,"etime":78},{"speaker":"Customer","text":"You too, bye.","stime":79,"etime":82}]'


# Get predictions
profanity_result = predictor.predict_profanity(conversation_json)
sensitive_result = predictor.predict_sensitive_data(conversation_json)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step


In [23]:
profanity_result

'Not Found'

In [24]:
sensitive_result

'found'

In [20]:
data = [{"speaker":"Agent","text":"Hello, this is Mark calling from XYZ Collections. How are you today?","stime":0,"etime":7},{"speaker":"Customer","text":"I amm okay, just busy. What is this about?","stime":8,"etime":12},{"speaker":"Agent","text":"I understand, I will be brief. I\'m calling regarding your outstanding balance with Quick Loans. Can you confirm your name for me?","stime":11,"etime":20},{"speaker":"Customer","text":"Sure, it\'s Lisa Green.","stime":21,"etime":24},{"speaker":"Agent","text":"Thank you, Lisa. You currently have a balance of $450. How would you like to proceed with the payment?","stime":23,"etime":32},{"speaker":"Customer","text":"I need more information before I decide.","stime":33,"etime":36},{"speaker":"Agent","text":"Of course! Your last payment was made in January, and your balance has been accumulating since then. Would you like to set up a payment plan?","stime":35,"etime":45},{"speaker":"Customer","text":"I might consider that. What are the options?","stime":46,"etime":50},{"speaker":"Agent","text":"We can do weekly or monthly payments. For example, a monthly payment of $150 would work for you.","stime":49,"etime":58},{"speaker":"Customer","text":"That sounds reasonable, but I need to think it over.","stime":59,"etime":62},{"speaker":"Agent","text":"I understand. Please let me know if you have any questions. Can I count on you to respond by the end of the week?","stime":63,"etime":72},{"speaker":"Customer","text":"Yes, I\'ll get back to you. Thank you.","stime":71,"etime":74},{"speaker":"Agent","text":"You\'re welcome, Lisa. Have a great day!","stime":75,"etime":78},{"speaker":"Customer","text":"You too, bye.","stime":79,"etime":82}]

conversation_str = json.dumps(data, ensure_ascii=False)

print(conversation_str)

[{"speaker": "Agent", "text": "Hello, this is Mark calling from XYZ Collections. How are you today?", "stime": 0, "etime": 7}, {"speaker": "Customer", "text": "I amm okay, just busy. What is this about?", "stime": 8, "etime": 12}, {"speaker": "Agent", "text": "I understand, I will be brief. I'm calling regarding your outstanding balance with Quick Loans. Can you confirm your name for me?", "stime": 11, "etime": 20}, {"speaker": "Customer", "text": "Sure, it's Lisa Green.", "stime": 21, "etime": 24}, {"speaker": "Agent", "text": "Thank you, Lisa. You currently have a balance of $450. How would you like to proceed with the payment?", "stime": 23, "etime": 32}, {"speaker": "Customer", "text": "I need more information before I decide.", "stime": 33, "etime": 36}, {"speaker": "Agent", "text": "Of course! Your last payment was made in January, and your balance has been accumulating since then. Would you like to set up a payment plan?", "stime": 35, "etime": 45}, {"speaker": "Customer", "text

In [21]:
# Predictor class and module

class CallAnalysisPredictor:
    """
    Class for making predictions on new data
    """
    def __init__(self):
        self.profanity_components = None
        self.sensitive_model = None
        self.sensitive_vectorizer = None
        
    def load_models(self):
        """
        Load trained models
        """
        # Load profanity model
        with open('profanity_model.pkl', 'rb') as f:
            self.profanity_components = pickle.load(f)
        
        # Load sensitive data model
        self.sensitive_model = tf.keras.models.load_model('sensitive_model.h5')
        
        with open('sensitive_vectorizer.pkl', 'rb') as f:
            self.sensitive_vectorizer = pickle.load(f)
    
    def preprocess_json_input(self, json_string):
        """
        Preprocess JSON input for prediction
        Args:
            json_string: JSON string of conversation
        Returns:
            processed_text: Clean text for prediction
        """
        try:
            conversation_list = json.loads(json_string)
            all_text = []
            
            for item in conversation_list:
                if 'text' in item:
                    all_text.append(item['text'])
            
            combined_text = " ".join(all_text)
            combined_text = combined_text.lower()
            combined_text = re.sub(r'[^\w\s]', ' ', combined_text)
            combined_text = re.sub(r'\s+', ' ', combined_text).strip()
            
            return combined_text
        except:
            return ""
    
    def predict_profanity(self, json_string):
        """
        Predict profanity in conversation
        """
        processed_text = self.preprocess_json_input(json_string)
        
        # Transform text
        text_tfidf = self.profanity_components['vectorizer'].transform([processed_text])
        
        # Predict
        prediction = self.profanity_components['model'].predict(text_tfidf)[0]
        
        # Convert back to label
        label_classes = self.profanity_components['label_encoder'].classes_
        result = label_classes[prediction]
        
        return result
    
    def predict_sensitive_data(self, json_string):
        """
        Predict sensitive data compliance violation
        """
        processed_text = self.preprocess_json_input(json_string)
        
        # Transform text
        text_tfidf = self.sensitive_vectorizer.transform([processed_text]).toarray()
        
        # Predict
        prediction = self.sensitive_model.predict(text_tfidf)[0][0]
        
        # Convert to binary classification
        result = "found" if prediction > 0.5 else "not found"
        
        return result